In [1]:
import pandas as pd


In [15]:

# === 1. 데이터 로드 ===
final_path = "perfume_final(부향률_용어_통일).csv"
hf_path = "C:\\Workspaces\\SKN14-Final-2Team\\Seong9\\deep_learning\\data_preprocessing\\perfumes_hf.csv"
output_path = "perfume_final(_with_gender).csv"



df_hf = pd.read_csv(
    hf_path, sep="|", encoding="utf-8-sig", engine="python",
    on_bad_lines="skip"
)
df_final = pd.read_csv(final_path, encoding="utf-8-sig")


print("perfume_final columns:", df_final.columns.tolist())
print("perfumes_hf columns:", df_hf.columns.tolist())


perfume_final columns: ['brand', 'name', 'eng_name', 'size_ml', 'price_krw', 'detail_url', 'concentration', 'main_accords', 'top_notes', 'middle_notes', 'base_notes', 'description', 'notes_score', 'season_score', 'day_night_score']
perfumes_hf columns: ['brand', 'name', 'family', 'subfamily', 'fragrances', 'ingredients', 'origin', 'gender', 'years', 'description', 'image_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,']


In [16]:
# === 2. 키 정규화 ===
def norm(s): return s.astype(str).str.strip().str.replace(r"\s+"," ",regex=True).str.lower()
df_final["__key"] = norm(df_final["eng_name"])
df_hf = df_hf.rename(columns={"name":"eng_name"})
df_hf["__key"] = norm(df_hf["eng_name"])

In [18]:
import numpy as np

# === 3. 이름 정규화 ===
def norm(s):
    return (s.astype(str).str.strip()
                     .str.replace(r"\s+"," ",regex=True)
                     .str.lower())

# === 4.키 만들기 (final, hf 모두) === 
df_final["__key"] = norm(df_final["eng_name"])
df_hf2 = df_hf.rename(columns={"name":"eng_name"}).copy()
df_hf2["__key"] = norm(df_hf2["eng_name"])

# === 5.hf를 이름별 대표 gender로 축약 (최빈값) === 
def mode_or_nan(s):
    x = s.dropna()
    if x.empty: return np.nan
    vc = x.value_counts()
    return sorted(vc[vc.eq(vc.max())].index)[0]

hf_key_gender = df_hf2.groupby("__key", as_index=False)["gender"].agg(mode_or_nan)

# === 6. many-to-one 병합 ===
df_merge = (df_final
            .merge(hf_key_gender, on="__key", how="left", validate="m:1")
            .drop(columns="__key"))

# === 7. 저장 ===
df_merge.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: {output_path}")
print(df_merge.head())


✅ 저장 완료: perfume_final(_with_gender).csv
  brand          name  eng_name  size_ml  price_krw  \
0   크리드   어벤투스 오 드 퍼퓸   Aventus       50     255000   
1   크리드   어벤투스 오 드 퍼퓸   Aventus      100     399220   
2  톰 포드  오드 우드 오 드 퍼퓸  Eau Wood       30     179000   
3  톰 포드  오드 우드 오 드 퍼퓸  Eau Wood       50     249000   
4    이솝     테싯 오 드 퍼퓸     Tecit       50     135000   

                                  detail_url concentration  \
0   https://www.bysuco.com/product/show/9370        오 드 퍼퓸   
1   https://www.bysuco.com/product/show/9370        오 드 퍼퓸   
2  https://www.bysuco.com/product/show/10716        오 드 퍼퓸   
3  https://www.bysuco.com/product/show/10716        오 드 퍼퓸   
4   https://www.bysuco.com/product/show/9970        오 드 퍼퓸   

         main_accords                top_notes      middle_notes  \
0          프루티 스위트 레더  베르가못 블랙 커런트 애플 레몬 핑크 페퍼  파인애플 패출리 모로칸 자스민   
1          프루티 스위트 레더  베르가못 블랙 커런트 애플 레몬 핑크 페퍼  파인애플 패출리 모로칸 자스민   
2       우디 오우드 웜 스파이시                      NaN     

In [19]:
# === 8. 병합물 데이터 확인 ===
df_gender = pd.read_csv(output_path, encoding="utf-8-sig")

print("perfumes_gender columns:", df_gender.columns.tolist())


missing_count = df_gender["gender"].isna().sum()
print("gender 결측치 개수:", missing_count)

perfumes_gender columns: ['brand', 'name', 'eng_name', 'size_ml', 'price_krw', 'detail_url', 'concentration', 'main_accords', 'top_notes', 'middle_notes', 'base_notes', 'description', 'notes_score', 'season_score', 'day_night_score', 'gender']
gender 결측치 개수: 935


In [26]:
# === Jupyter 전용: 브랜드 매핑 + 단계적 병합 + 경량 퍼지 매칭 ===
# 1) 아래 경로 3개만 여러분 환경에 맞게 수정하세요.
USER_CSV = r"C:\Workspaces\SKN14-Final-2Team\Seong9\perfume_final(_with_gender).csv"
REF_CSV  = r"C:\Workspaces\SKN14-Final-2Team\Seong9\deep_learning\data_preprocessing\perfumes_hf.csv"
OUT_DIR  = r"C:\Workspaces\SKN14-Final-2Team\Seong9"

# 2) 퍼지 매칭 임계값 (원하면 조절)
FUZZY_AUTO  = 0.90  # 이 이상은 자동 반영
FUZZY_REVIEW_FLOOR = 0.80  # 이 이상~AUTO 미만은 검토 리스트로

import os, re, unicodedata, pandas as pd, numpy as np
from difflib import SequenceMatcher
from pathlib import Path

# ---------- 유틸 ----------
def ensure_path(p: str) -> Path:
    path = Path(p)
    if not path.exists():
        # 폴더/파일 점검 힌트
        raise FileNotFoundError(f"경로 없음: {path}\n- 디렉토리 존재? {path.parent} -> {path.parent.exists()}\n- 파일명(띄어쓰기/괄호/대소문자) 다시 확인하세요.")
    return path

def sniff_sep_by_head(path: Path, enc="utf-8-sig"):
    with open(path, "r", encoding=enc, errors="replace") as f:
        head = f.readline()
    candidates = [("|", head.count("|")), (",", head.count(",")), ("\t", head.count("\t")), (";", head.count(";"))]
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[0][0] if candidates and candidates[0][1] > 0 else ","

def load_any_csv(path: Path, prefer_pipe=False):
    # utf-8-sig 우선, 실패시 cp949 시도
    try:
        sep = "|" if prefer_pipe else sniff_sep_by_head(path, "utf-8-sig")
        return pd.read_csv(path, sep=sep, encoding="utf-8-sig", engine="python", on_bad_lines="skip")
    except Exception:
        sep = "|" if prefer_pipe else sniff_sep_by_head(path, "cp949")
        return pd.read_csv(path, sep=sep, encoding="cp949", engine="python", on_bad_lines="skip")


def strip_accents(s: str) -> str:
    s = unicodedata.normalize('NFKD', s)
    return ''.join(ch for ch in s if not unicodedata.combining(ch))

def clean_name(x: str) -> str:
    if pd.isna(x): return ""
    s = str(x)
    s = strip_accents(s)
    # 괄호 내용/용량/농도 표기 제거
    s = re.sub(r"[\(\[\{].*?[\)\]\}]", " ", s)
    s = re.sub(r"\b\d+(\.\d+)?\s*(ml|mL|ML|oz|OZ|fl\.?\s*oz)\b", " ", s)
    s = re.sub(r"\b(extrait|edp|edt|edc|parfum|intense|eau\s*de\s*parfum|eau\s*de\s*toilette|eau\s*de\s*cologne)\b", " ", s, flags=re.IGNORECASE)
    # 안전 문자만 남기고 공백 정리
    s = re.sub(r"[^\w\s\-&/']", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def mode_or_nan(s: pd.Series):
    x = s.dropna()
    if x.empty: return np.nan
    vc = x.value_counts()
    return sorted(vc[vc.eq(vc.max())].index)[0]  # 동률이면 사전순

def token_overlap_score(a: str, b: str) -> float:
    ta, tb = set(a.split()), set(b.split())
    if not ta or not tb: return 0.0
    inter, union = len(ta & tb), len(ta | tb)
    return inter/union

# ---------- 실행 ----------
USER_CSV = str(ensure_path(USER_CSV))
REF_CSV  = str(ensure_path(REF_CSV))
OUT_DIR  = str(ensure_path(OUT_DIR))
out_csv   = str(Path(OUT_DIR) / "perfume_final_gender_filled_all.csv")
review_csv= str(Path(OUT_DIR) / "perfume_gender_review_needed_all.csv")

# 사용자/레퍼런스 로드 (레퍼런스는 파이프 가능성 높음)
df_user = load_any_csv(Path(USER_CSV))
df_ref  = load_any_csv(Path(REF_CSV), prefer_pipe=True)

# 레퍼런스 헤더 정리(마지막 열 이름에 콤마 꼬임 방지)
df_ref.columns = [c.split(",")[0].strip() for c in df_ref.columns]

# 필수 컬럼 확인/선택
def pick(df, cands, tag):
    for c in cands:
        if c in df.columns: return c
    raise KeyError(f"[{tag}] 필요한 컬럼을 찾지 못함. 후보: {cands}\n현재 컬럼: {df.columns.tolist()}")

user_brand_col = pick(df_user, ["brand","brand_eng","brand_en"], "user")
user_name_col  = pick(df_user, ["eng_name","name_eng","name_en","name","name_perfume","english_name"], "user")
ref_brand_col  = pick(df_ref,  ["brand","brand_eng","brand_en"], "ref")
ref_name_col   = pick(df_ref,  ["name","name_eng","name_en","name_perfume","english_name"], "ref")
if "gender" not in df_ref.columns:
    raise KeyError(f"[ref] 'gender' 컬럼 없음. 현재 컬럼: {df_ref.columns.tolist()}")

# --- 브랜드 매핑(초안; 필요시 아래 dict에 추가) ---
brand_map = {
    "샤넬": "Chanel", "디올": "Dior", "구찌": "Gucci", "에르메스": "Hermès",
    "메종 마르지엘라": "Maison Margiela", "톰 포드": "Tom Ford",
    "프레데릭 말": "Frédéric Malle", "딥티크": "Diptyque", "조 말론": "Jo Malone",
    "킬리안": "Kilian", "바이레도": "Byredo", "이솝": "Aesop", "르 라보": "Le Labo",
    "겔랑": "Guerlain", "입생로랑": "Yves Saint Laurent", "아쿠아 디 파르마": "Acqua di Parma",
    "몽블랑": "Montblanc", "프라다": "Prada", "돌체앤가바나": "Dolce & Gabbana",
    "로에베": "Loewe", "메종 프란시스 커정": "Maison Francis Kurkdjian",
}
df_user["brand_mapped"] = df_user[user_brand_col].map(brand_map).fillna(df_user[user_brand_col])

# 정규화 키 준비
df_user["_brand_norm"] = df_user["brand_mapped"].map(clean_name)
df_user["_name_norm"]  = df_user[user_name_col].map(clean_name)
df_user["_key0"]       = (df_user["_brand_norm"] + "||" + df_user["_name_norm"])

df_ref["_brand_norm"] = df_ref[ref_brand_col].map(clean_name)
df_ref["_name_norm"]  = df_ref[ref_name_col].map(clean_name)
df_ref["_key0"]       = (df_ref["_brand_norm"] + "||" + df_ref["_name_norm"])

# === 안전 병합: 사용자 gender 보존 → 레퍼런스 gender는 별칭으로 머지 ===

# 0) 사용자쪽에 gender가 이미 있으면 보존
if "gender" in df_user.columns:
    df_user = df_user.rename(columns={"gender": "gender_orig"})

# 1) brand+name 정확 매칭 (ref 쪽 gender를 'gender_ref'로 명시)
ref_gender_by_key = (
    df_ref.groupby("_key0", as_index=False)["gender"]
          .agg(mode_or_nan)
          .rename(columns={"gender":"gender_ref"})
)
m1 = df_user.merge(ref_gender_by_key, on="_key0", how="left", validate="m:1")

# 2) name-only 정확 매칭(잔여만) — 역시 별칭으로 받아서 채움
ref_gender_by_name = (
    df_ref.groupby("_name_norm", as_index=False)["gender"]
          .agg(mode_or_nan)
          .rename(columns={"gender":"gender_ref2"})
)

# 채울 그릇 만들기: 우선순위는 key → name
m1["gender_filled"] = m1["gender_ref"]
mask = m1["gender_filled"].isna()
m1.loc[mask, "gender_filled"] = (
    m1.loc[mask]
      .merge(ref_gender_by_name, on="_name_norm", how="left")["gender_ref2"]
      .values
)

# 3) (선택) 퍼지 매칭을 한다면, m1["gender_filled"] 가 NaN인 행만 대상으로 진행하여
#    m1.loc[apply_idx, "gender_filled"] = ... 형태로 채워 넣으세요.

# 4) 최종 정리: 최종 gender 컬럼 확정
#    - 기존 사용자 gender가 있었으면 우선 보존본 사용, 없으면 filled 사용
if "gender_orig" in m1.columns:
    # 기존값 우선, 없으면 새로 채운 값
    m1["gender"] = m1["gender_orig"].where(m1["gender_orig"].notna(), m1["gender_filled"])
else:
    m1["gender"] = m1["gender_filled"]

# 5) 보조 컬럼 정리
drop_cols = ["gender_ref", "gender_ref2", "gender_filled"]
m1_final = m1.drop(columns=[c for c in drop_cols if c in m1.columns], errors="ignore")

# 6) 저장
out_csv = str(Path(OUT_DIR) / "perfume_final_gender_filled_all.csv")
m1_final.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", out_csv)

# 요약
total = len(m1_final)
filled = m1_final["gender"].notna().sum()
print(f"채워진 gender: {filled}/{total} ({filled/total:.1%})")

✅ 저장 완료: C:\Workspaces\SKN14-Final-2Team\Seong9\perfume_final_gender_filled_all.csv
채워진 gender: 643/1395 (46.1%)


In [28]:
import pandas as pd, numpy as np, re, unicodedata
from difflib import SequenceMatcher
from pathlib import Path

# === 파일 경로 ===
USER_CSV = r"C:\Workspaces\SKN14-Final-2Team\Seong9\perfume_final(_with_gender).csv"
REF_CSV  = r"C:\Workspaces\SKN14-Final-2Team\Seong9\deep_learning\data_preprocessing\perfumes_hf.csv"
OUT_DIR  = r"C:\Workspaces\SKN14-Final-2Team\Seong9\deep_learning\data_preprocessing"

OUT_CSV   = str(Path(OUT_DIR) / "perfume_final_gender_filled_all.csv")
REVIEW_CSV= str(Path(OUT_DIR) / "perfume_gender_review_needed_all.csv")

# === 유틸 함수 ===
def strip_accents(s: str) -> str:
    s = unicodedata.normalize('NFKD', s)
    return ''.join(ch for ch in s if not unicodedata.combining(ch))

def clean_name(x: str) -> str:
    if pd.isna(x): return ""
    s = str(x)
    s = strip_accents(s)
    s = re.sub(r"[\(\[\{].*?[\)\]\}]", " ", s)
    s = re.sub(r"\b\d+(\.\d+)?\s*(ml|mL|ML|oz|OZ|fl\.?\s*oz)\b", " ", s)
    s = re.sub(r"\b(extrait|edp|edt|edc|parfum|intense|eau\s*de\s*parfum|eau\s*de\s*toilette|eau\s*de\s*cologne)\b",
               " ", s, flags=re.IGNORECASE)
    s = re.sub(r"[^\w\s\-&/']", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def norm_key(brand, name):
    return f"{clean_name(str(brand))}||{clean_name(str(name))}"

def mode_or_nan(s: pd.Series):
    x = s.dropna()
    if x.empty: return np.nan
    vc = x.value_counts()
    return sorted(vc[vc.eq(vc.max())].index)[0]

def token_overlap_score(a: str, b: str) -> float:
    ta, tb = set(a.split()), set(b.split())
    if not ta or not tb: return 0.0
    inter, union = len(ta & tb), len(ta | tb)
    return inter/union

# === 데이터 로드 ===
df_user = pd.read_csv(USER_CSV, encoding="utf-8-sig")
df_ref  = pd.read_csv(REF_CSV, sep="|", encoding="utf-8-sig", engine="python", on_bad_lines="skip")
df_ref.columns = [c.split(",")[0].strip() for c in df_ref.columns]

# === 사용자 gender 보존 ===
if "gender" in df_user.columns:
    df_user = df_user.rename(columns={"gender": "gender_orig"})

# === 컬럼 지정 ===
user_brand_col = "brand"
user_name_col  = "eng_name" if "eng_name" in df_user.columns else "name_eng"
ref_brand_col  = "brand"
ref_name_col   = "name"

# === 정규화 키 ===
df_user["_brand_norm"] = df_user[user_brand_col].map(clean_name)
df_user["_name_norm"]  = df_user[user_name_col].map(clean_name)
df_user["_key0"]       = df_user.apply(lambda r: norm_key(r[user_brand_col], r[user_name_col]), axis=1)

df_ref["_brand_norm"] = df_ref[ref_brand_col].map(clean_name)
df_ref["_name_norm"]  = df_ref[ref_name_col].map(clean_name)
df_ref["_key0"]       = df_ref.apply(lambda r: norm_key(r[ref_brand_col], r[ref_name_col]), axis=1)

# === 1) brand+name 정확 매칭 ===
ref_gender_by_key = (
    df_ref.groupby("_key0", as_index=False)["gender"]
          .agg(mode_or_nan)
          .rename(columns={"gender":"gender_ref"})
)
m1 = df_user.merge(ref_gender_by_key, on="_key0", how="left", validate="m:1")
m1["gender_filled"] = m1["gender_ref"]

# === 2) name-only 매칭 ===
ref_gender_by_name = (
    df_ref.groupby("_name_norm", as_index=False)["gender"]
          .agg(mode_or_nan)
          .rename(columns={"gender":"gender_ref2"})
)
mask = m1["gender_filled"].isna()
m1.loc[mask, "gender_filled"] = (
    m1.loc[mask].merge(ref_gender_by_name, on="_name_norm", how="left")["gender_ref2"].values
)

# === 3) 퍼지 매칭(브랜드 블록 + 후보 제한) ===
FUZZY_AUTO = 0.90
FUZZY_REVIEW_FLOOR = 0.80
remain = m1[m1["gender_filled"].isna()].copy()
review_rows = []

if not remain.empty:
    ref_blocks = {b: grp for b, grp in df_ref.groupby("_brand_norm")}
    for idx, row in remain.iterrows():
        b = row["_brand_norm"]
        q = row["_name_norm"]
        cand_grp = ref_blocks.get(b, df_ref)
        cand_names = cand_grp["_name_norm"].dropna().unique().tolist()
        if not cand_names:
            continue
        prelim = sorted(((cand, token_overlap_score(q, cand)) for cand in cand_names),
                        key=lambda x: x[1], reverse=True)[:40]  # 상위 40개만 상세검사
        best_name, best_score = None, 0.0
        for cand, _ in prelim:
            sc = SequenceMatcher(None, q, cand).ratio()
            if sc > best_score:
                best_score, best_name = sc, cand
        if best_name and best_score >= FUZZY_REVIEW_FLOOR:
            g = mode_or_nan(cand_grp[cand_grp["_name_norm"] == best_name]["gender"])
            if best_score >= FUZZY_AUTO:
                m1.at[idx, "gender_filled"] = g
            else:
                review_rows.append({
                    "brand_input": row[user_brand_col],
                    "eng_name_input": row[user_name_col],
                    "suggested_name_norm": best_name,
                    "suggested_gender": g,
                    "similarity": round(best_score, 4)
                })

# === 4) 최종 gender 합치기 ===
if "gender_orig" in m1.columns:
    m1["gender_final"] = m1["gender_orig"].where(m1["gender_orig"].notna(), m1["gender_filled"])
else:
    m1["gender_final"] = m1["gender_filled"]

# === 5) 저장 ===
final_out = m1.drop(columns=["_key0","gender_ref","gender_ref2","gender_filled"], errors="ignore")
final_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
pd.DataFrame(review_rows).to_csv(REVIEW_CSV, index=False, encoding="utf-8-sig")

print("✅ 저장 완료:", OUT_CSV)
print("검토 필요 목록:", REVIEW_CSV)
print("총 행:", len(final_out))
print("최종 gender 채움 비율:", final_out["gender_final"].notna().mean())


✅ 저장 완료: C:\Workspaces\SKN14-Final-2Team\Seong9\deep_learning\data_preprocessing\perfume_final_gender_filled_all.csv
검토 필요 목록: C:\Workspaces\SKN14-Final-2Team\Seong9\deep_learning\data_preprocessing\perfume_gender_review_needed_all.csv
총 행: 1395
최종 gender 채움 비율: 0.5519713261648745
